# Lab 7 —  SDM optimization

##### Tomasz Ogiołda

## Overview

This notebook utilizes numba package to improve performance of SCM coagulation solver. Numba, thank to JIT compilation and parallelization gave huge performance gain in comparison to standard non-parallel method.

In [2]:
from types import SimpleNamespace
import numpy as np
import scipy
import numba

In [3]:
PARAMS_PHYS = SimpleNamespace(
    n0=2**23,
    dv_m3 = 1e6,
    x0_kg=1.192e-13,
    b_per_s = 1500,
)
PARAMS_PHYS.dist = scipy.stats.expon(loc=0, scale=PARAMS_PHYS.x0_kg)
PARAMS_PHYS.norm = PARAMS_PHYS.n0 * PARAMS_PHYS.dv_m3

PARAMS_COMP = SimpleNamespace(
    n_part = 2**11,
    t_max_s = 3600,
    n_step = 1800,
)
PARAMS_COMP.dt_s = PARAMS_COMP.t_max_s / PARAMS_COMP.n_step

PARAMS_BINS = SimpleNamespace(
    min_x = -12,
    max_x = -5,
    count = 70
)

RNG = np.random.default_rng(seed=44)

In [4]:
def sample(*, params_comp, params_phys, rng):
    """ randomly samples a particle population using constant-multiplicity,
    uniform-mass and uniform-log-mass schemes and returns a dictionary
    of three simulation state, each composed of 'mass' and 'mult' arrays """
    u01 = rng.uniform(0, 1, size=params_comp.n_part)
    uniform_sampling_range = [params_phys.dist.ppf(q) for q in (.001, .999)]
    x_uniform_linx = (
        uniform_sampling_range[0]
        + u01 * (uniform_sampling_range[1] - uniform_sampling_range[0])
    )
    x_uniform_logx = np.exp(
        np.log(uniform_sampling_range[0])
        + u01 * (np.log(uniform_sampling_range[1]) - np.log(uniform_sampling_range[0]))
    )
    return {
        k: {
            'mass': v['x'],
            'mult': np.round(v['y'] * params_phys.norm).astype(int),
        }
        for k,v in
        {
            'sampling: uniform random in x': {
                'x': x_uniform_linx,
                'y': params_phys.dist.pdf(x_uniform_linx) \
                  * (uniform_sampling_range[1] - uniform_sampling_range[0]) \
                  / params_comp.n_part,
            },
            'sampling: uniform random in ln(x)': {
                'x': x_uniform_logx,
                'y': params_phys.dist.pdf(x_uniform_logx) \
                  * (np.log(uniform_sampling_range[1]) - np.log(uniform_sampling_range[0])) \
                  / (params_comp.n_part / x_uniform_logx),
            },
            'sampling: constant multiplicity': {
                'x': params_phys.dist.ppf(u01),
                'y': np.full(shape=params_comp.n_part, fill_value=1 / params_comp.n_part),
            }
        }.items()
    }

PARTICLES = sample(params_phys=PARAMS_PHYS, params_comp=PARAMS_COMP, rng=RNG)

In [5]:
def x_of_mass(mass):
    """ defines the plot x coordinate as a funciton of particle mass """
    return np.log(mass) / 3

def mass_of_x(coord):
    """ computes mass back from the plot x coordinate """
    return np.exp(3 * coord)

def kernel(mass_1, mass_2, coeff):
    """ additive coagulation kernel """
    return coeff * (mass_1 + mass_2)

def analytic_solution(mass_kg, time_s, params_phys):
    """ Golovin's analytic solution to Smoluchowski coagulation equation
    for additive kernel and exponential initial condition """
    tau = 1 - np.exp(-params_phys.n0 * params_phys.b_per_s * params_phys.x0_kg * time_s)
    sqrt_tau = np.sqrt(tau)
    return (
        (1 - tau) / (mass_kg * sqrt_tau)
        * scipy.special.ive(1, 2 * mass_kg / params_phys.x0_kg * sqrt_tau)  # pylint: disable=no-member
        * np.exp(-(1 + tau - 2 * sqrt_tau) * mass_kg / params_phys.x0_kg)
    )

In [6]:
@numba.jit
def kernel(mass_1, mass_2, coeff):
    """ additive coagulation kernel """
    return coeff * (mass_1 + mass_2)


@numba.jit(parallel=True)
def sdm_optimized(*, pairs, u01, mult, mass, coeff, dt, dv):
    """ performs Monte-Carlo coagulation using the SDM algorithm """
    for alpha in numba.prange(pairs.shape[0]):
        (j, k) = pairs[alpha]

        p_jk = kernel(mass[j], mass[k], coeff) * dt / dv
        p_s_jk = max(mult[j], mult[k]) * p_jk
        n = len(mult)
        p_alpha = p_s_jk * n * (n - 1) / (2 * len(pairs))

        if u01[alpha] < p_alpha:
            if mult[j] < mult[k]:
                j, k = k, j

            if mult[j] != mult[k]:
                mult[j] -= mult[k]
                mass[k] += mass[j]

            else:
                mult[k] = mult[j] - mult[j] // 2
                mult[j] = mult[j] // 2
                mass[k] += mass[j]
                mass[j] = mass[k]


def sdm(pairs, u01, mult, mass, coeff, dt, dv):
    """ performs Monte-Carlo coagulation using the SDM algorithm """
    p_scale = len(mult) * (len(mult) - 1) / 2 / len(pairs)
    for alpha, (j, k) in enumerate(pairs):
        if mult[j] < mult[k]:
            j, k = k, j
        p_alpha = mult[j] * p_scale * kernel(mass[j], mass[k], coeff) * dt / dv
        gamma = p_alpha // 1 + (p_alpha - p_alpha // 1) > u01[alpha]
        if gamma != 0:
            gamma = min(gamma, (mult[j] / mult[k]) // 1)
            if mult[j] - gamma * mult[k] > 0:
                mult[j] -= gamma * mult[k]
                mass[k] += gamma * mass[j]
            else:
                mult[j] = mult[k] // 2
                mult[k] -= mult[j]
                mass[k] += gamma * mass[j]
                mass[j] = mass[k]

In [7]:
def simulate(*, params_phys, params_comp, particles, rng, sdm=sdm):
    """ does simulation for all sampling variants (each variant using the same shuffled numbers) """
    n_pairs = params_comp.n_part // 2
    for _ in range(params_comp.n_step):
        non_overlapping_pairs = rng.permutation(params_comp.n_part)[: 2 * n_pairs].reshape(-1, 2)
        u01 = rng.uniform(0, 1, n_pairs)
        for part in particles.values():
            sdm(
                pairs=non_overlapping_pairs,
                u01=u01,
                coeff=params_phys.b_per_s,
                dt=params_comp.dt_s,
                dv=params_phys.dv_m3,
                **part
            )

In [8]:
import time, copy

def measure_perf(sdm):
    BENCH_RNG = np.random.default_rng(seed=44)

    start_time = time.perf_counter()

    part_cpy = copy.deepcopy(PARTICLES)

    simulate(
        particles=part_cpy,
        params_phys=PARAMS_PHYS,
        params_comp=PARAMS_COMP,
        rng=BENCH_RNG,
        sdm=sdm
    )

    end_time = time.perf_counter()
    duration = end_time - start_time

    return {
        'duration': duration,
        'particles': part_cpy
    }


raw_sdm = measure_perf(sdm)
optimized_sdm = measure_perf(sdm_optimized)

## Running time comparison

### Not optimized version (sec)

In [9]:
raw_sdm['duration']

57.361841337000214

### Optimized version (sec)

In [10]:
optimized_sdm['duration']

0.9578236029999516

## Result correctness verification

In [11]:
import pickle

assert pickle.dumps(raw_sdm['particles']) == pickle.dumps(optimized_sdm['particles'])